In [1]:
import os
import glob
import json 
import time
import math
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tensorboardX import SummaryWriter


from byol_pytorch import BYOL
from torchvision import models

In [2]:
train_path = "/data/dlcv/hw4/office/train"
valid_path = "/data/dlcv/hw4/office/val"
train_csv = "/data/dlcv/hw4/office/train.csv"
valid_csv = "/data/dlcv/hw4/office/val.csv"
label2id_path = "./label2id.json"
ckpt_path = "./ckpt/finetune"
os.makedirs(ckpt_path, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(4)
print('Device used:', device)

img_size = 128
train_bz = 64
valid_bz = 200

Device used: cuda


In [3]:
df = pd.read_csv(train_csv)
print(df.loc[df['filename']=="Couch00015.jpg"]['label'].to_list())
print(df.index[df['filename']=="Couch00015.jpg"].to_list())
print(df['label'][0])

['Couch']
[0]
Couch


In [4]:
with open("/data/jhccc/dlcv/hw4-jhc659/label2id.json", 'r') as j:
    label2id = json.loads(j.read())
print(type(label2id))

<class 'dict'>


In [5]:
# label = df.loc[df['filename']=="Couch00015.jpg"].label
label = df.loc[2].label

print(label)
# print(label2id[label[0]])
# print(label2id)

Refrigerator


In [6]:
class dataset(Dataset):
    def __init__(self, inputPath, csvPath, label2idPath, transform=None):
        self.inputPath = inputPath
        self.transform = transform
        with open(label2idPath, 'r') as j:
            self.label2id = json.loads(j.read())
        self.inputName = []
        df = pd.read_csv(csvPath)
        for i in range(len(df)):
            self.inputName.append((df.loc[i].filename, self.label2id[df.loc[i].label]))
        print(self.inputName[0])
        
    def __getitem__(self, index):
        img = Image.open(os.path.join(self.inputPath, self.inputName[index][0]))
        if self.transform:
            img = self.transform(img)
        id = self.inputName[index][1]
        return img, id

    def __len__(self):
        return len(self.inputName)

img_transform = transforms.Compose([
    transforms.Resize(size=128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

def save_checkpoint(ckpt_path, model, optimizer):
    state = {'model_state_dict': model.state_dict(),
             'optimizer_state_dict': optimizer.state_dict(),}
    torch.save(state, ckpt_path)

def load_checkpoint(ckpt_path, device):
    ckpt = torch.load(ckpt_path, map_location=device)
    return ckpt

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [7]:
trainDS = dataset(inputPath=train_path, csvPath=train_csv, label2idPath=label2id_path, transform=img_transform)
trainLoader = DataLoader(dataset=trainDS, batch_size=train_bz, shuffle=True, num_workers=4)
validDS = dataset(inputPath=valid_path, csvPath=valid_csv, label2idPath=label2id_path, transform=img_transform)
validLoader = DataLoader(dataset=validDS, batch_size=valid_bz, shuffle=False, num_workers=1)
print('# images in trainset:', len(trainDS))
print('# images in validset:', len(validDS))

# dataiter = iter(validLoader)
# images, labels = dataiter.next()
# # print(labels)
# print('Image tensor in each batch:', images.shape, images.dtype)
# print('Label tensor in each batch:', labels.shape, labels.dtype)

('Couch00015.jpg', 13)
('Fork00005.jpg', 24)
# images in trainset: 3951
# images in validset: 406


In [8]:
class settingA(nn.Module):
    def __init__(self, ckpt_path=None) -> None:
        super().__init__()
        self.resnet = models.resnet50(weights=None)
        self.classifier = nn.Sequential(
            nn.Linear(self.resnet.fc.out_features, 65)
        )
    def forward(self, x):
        x = self.resnet(x)
        return self.classifier(x)

In [9]:
# trainning
def train(model, epochs):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,100], gamma=0.9)
    criterion = nn.CrossEntropyLoss()
    best_acc = 0.

    for epoch in range(epochs):
        model.train()        
        train_loss, valid_loss = 0, 0
        train_acc, valid_acc = 0, 0
        for i, (img, label) in enumerate(trainLoader):
            img, label = img.to(device), label.to(device)
            optimizer.zero_grad()
            output = model(img)
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            train_acc += pred.eq(label.view_as(pred)).sum().item()
            loss = criterion(output, label)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()                
        scheduler.step()
        train_loss /= (i+1)
        train_acc /= len(trainLoader.dataset)
        print("Epoch: {:02}".format(epoch))
        print(" |train_loss: {:6f}, train_acc: {:.2%}".format(train_loss, train_acc))
        model.eval()
        with torch.no_grad(): # This will free the GPU memory used for back-prop
            for i, (img, label) in enumerate(validLoader):
                img, label = img.to(device), label.to(device)
                output = model(img)
                valid_loss += criterion(output, label).item() # sum up batch loss
                pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
                valid_acc += pred.eq(label.view_as(pred)).sum().item()
                # resultClass = torch.argmax(output, dim=1)
                # acc += (resultClass == target).sum()
            valid_loss /= (i+1)
            valid_acc /= len(validLoader.dataset)

        if valid_acc > best_acc:
            save_checkpoint(os.path.join(ckpt_path, "settingA_best.pth"), model, optimizer)
            print("| Save checkpoint for epoch {}".format(epoch+1))
            
            best_acc = valid_acc
        save_checkpoint(os.path.join(ckpt_path, "settingA_last.pth"), model, optimizer)
        print(" |valid_loss: {:6f}, train_acc: {:.2%}".format(valid_loss, valid_acc))
        

In [10]:
model = settingA().to(device)
train(model, 100)

Epoch: 00
 |train_loss: 5.881514, train_acc: 1.92%
| Save checkpoint for epoch 1
 |valid_loss: 6.023097, train_acc: 2.71%
Epoch: 01
 |train_loss: 4.144563, train_acc: 2.28%
 |valid_loss: 4.162073, train_acc: 2.22%
Epoch: 02
 |train_loss: 4.081208, train_acc: 3.47%
| Save checkpoint for epoch 3
 |valid_loss: 4.103574, train_acc: 5.91%
Epoch: 03
 |train_loss: 3.987463, train_acc: 4.88%
 |valid_loss: 3.887041, train_acc: 4.68%
Epoch: 04
 |train_loss: 3.919579, train_acc: 5.47%
 |valid_loss: 4.189121, train_acc: 5.91%
Epoch: 05
 |train_loss: 3.872905, train_acc: 6.28%
| Save checkpoint for epoch 6
 |valid_loss: 3.929351, train_acc: 8.62%
Epoch: 06
 |train_loss: 3.810718, train_acc: 7.29%
 |valid_loss: 3.773169, train_acc: 8.37%
Epoch: 07
 |train_loss: 3.740709, train_acc: 8.15%
| Save checkpoint for epoch 8
 |valid_loss: 3.785695, train_acc: 9.11%
Epoch: 08
 |train_loss: 3.696111, train_acc: 9.34%
 |valid_loss: 3.824679, train_acc: 7.14%
Epoch: 09
 |train_loss: 3.647703, train_acc: 10.15%


KeyboardInterrupt: 